## Using RACE Dataset with LLMs Locally
The model initially used is Mistral Instruct with 7 billion parameters which is compressed with Quantization 4

In [1]:
import openai
import pandas as pd
from datasets import load_dataset
import os

In [2]:
openai.api_base = "http://localhost:1234/v1"
openai.api_key = ""

In [3]:
dataset = load_dataset("race", "all")

In [5]:
dataset.shape

{'test': (4934, 5), 'train': (87866, 5), 'validation': (4887, 5)}

In [7]:
df = pd.DataFrame(dataset['train']).sample(n=10)

In [8]:
df

,example_id,article,answer,question,options
42190,high10153.txt,"During the seasons of fine weather, kids can p...",B,The indoor treasure hunt is an activity in whi...,"[would like to get much treasure, are eager to..."
39451,high2264.txt,"Dear editor,\nMary is my friend and she is a v...",D,What kind of person is Mary?,"[Friendly and beautiful., Kind and careful., C..."
84232,middle2608.txt,Liu Hong is from China. She is a nice girl. He...,C,How many people are there in Liu Hong's family?,"[Three, Four, Five, Six]"
4003,high5724.txt,Researchers claimed that waitresses who wear r...,B,What is the main idea of the passage?,[People tip differently if waitresses wear dif...
85415,middle3536.txt,Most people will rest and relax when they are ...,C,Grandma Moses starts to paint pictures _ .,"[when her daughter asks her to do that, when s..."
65823,middle7127.txt,Good afternoon and welcome to England. We hope...,D,"From the speech, we can learn that in England, .","[you can throw rubbish in the street, the traf..."
85920,middle761.txt,Do your kids ask a lot of questions? If they d...,C,How much will you pay if you don't like the fi...,"[$11.95., $1.195., Nothing., $10.]"
3981,high6771.txt,Einstein's connection with the politics of the...,B,We can learn from the passage that Einstein _,"[believed in God, was proud of being Jewish, s..."
18491,high152.txt,Everyone has got two personalities --the one t...,C,Point out which sentence is used to show the p...,"[is always cheerful, lives for tomorrow, can't..."
76647,middle8117.txt,Fire can help people in many ways. But it can ...,D,Which of the following is TRUE?,[Someone started a fire in Heilongjiang Provin...


In [17]:
def getAnswerFromModel(row):
    article = row['article']
    question = row['question']
    options = row['options']
    
    response = openai.ChatCompletion.create(model="local-model", messages=[
        {'role': "system", 'content': "Perform Extractive Question Answering from the article. Select the correct option in form of A, B, C, or D only." },
        {'role': "user", 'content': f"Article: {article}\nQuestion: {question}\nOptions: {', '.join(options)}\nAnswer:?"}
    ])
    
    return response.choices[0].message.content

In [18]:
# Testing on one row
testAnswer = getAnswerFromModel(df.loc[42190])
testAnswer

'are eager to find something.'

In [19]:
df["Local Mistral Instruct 7B Q4_K_S"] = df.apply(getAnswerFromModel, axis=1)

In [20]:
df

,example_id,article,answer,question,options,Local Mistral Instruct 7B Q4_K_S
42190,high10153.txt,"During the seasons of fine weather, kids can p...",B,The indoor treasure hunt is an activity in whi...,"[would like to get much treasure, are eager to...",are eager to find something.
39451,high2264.txt,"Dear editor,\nMary is my friend and she is a v...",D,What kind of person is Mary?,"[Friendly and beautiful., Kind and careful., C...",Careful and childish.
84232,middle2608.txt,Liu Hong is from China. She is a nice girl. He...,C,How many people are there in Liu Hong's family?,"[Three, Four, Five, Six]","Answer: Five\nExplanation:\nFrom the article, ..."
4003,high5724.txt,Researchers claimed that waitresses who wear r...,B,What is the main idea of the passage?,[People tip differently if waitresses wear dif...,B) Waitresses who wear red get more tips than ...
85415,middle3536.txt,Most people will rest and relax when they are ...,C,Grandma Moses starts to paint pictures _ .,"[when her daughter asks her to do that, when s...",when she's 70.
65823,middle7127.txt,Good afternoon and welcome to England. We hope...,D,"From the speech, we can learn that in England, .","[you can throw rubbish in the street, the traf...",You may not buy cigarettes or tobacco unless y...
85920,middle761.txt,Do your kids ask a lot of questions? If they d...,C,How much will you pay if you don't like the fi...,"[$11.95., $1.195., Nothing., $10.]","The correct answer is ""Nothing."" According to ..."
3981,high6771.txt,Einstein's connection with the politics of the...,B,We can learn from the passage that Einstein _,"[believed in God, was proud of being Jewish, s...","belied in God, was proud of being Jewish, sugg..."
18491,high152.txt,Everyone has got two personalities --the one t...,C,Point out which sentence is used to show the p...,"[is always cheerful, lives for tomorrow, can't...","lives for today. \nTherefore, the correct answ..."
76647,middle8117.txt,Fire can help people in many ways. But it can ...,D,Which of the following is TRUE?,[Someone started a fire in Heilongjiang Provin...,Someone started a fire in Heilongjiang Provinc...


In [21]:
df.to_csv("RACE_Local_MistralInstruct7BQ4.csv", index=False)